In [2]:
import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')
# df2 = pd.read_csv('Syngenta/Syngenta_2017/Region_dataset.csv')

In [ ]:
# SEE SOME OPTIONAL INFORMATION ABOUT THE DATAFRAME

# print(df.head())
# print(df.describe())
print(df)
print(df.Variety.unique(), "\nthere are ", len(df.Variety.unique()), " unique varieties")

In [8]:
# OPTIONAL CORRELATION CHECKER

from sklearn import preprocessing

# x = df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df = pandas.DataFrame(x_scaled)

#these were selected mainly due to their dtype
df_potential_columns = df.drop(['Experiment', 'Location', 'Planting date', 'Check Yield', 'Yield difference', 'Latitude', 'Longitude', 'Variety', 'PI'], axis=1).columns
# potential_feature_columns = ['Location', 'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class', 'CEC', 'pH', 'Clay', 'Silt', 'Sand', 'Area']
# print(potential_feature_columns)
potential_output_column = df.Yield

new_df = df.loc[:, df_potential_columns]
print(new_df.columns)

potential_X = df.loc[:, df_potential_columns].values
min_max_scaler = preprocessing.MinMaxScaler()
# normalize = preprocessing.normalize()
potential_y = potential_output_column
x_scaled = min_max_scaler.fit_transform(potential_X)
y_scaled = preprocessing.normalize(potential_y.reshape(-1,1))
df2 = pd.DataFrame(x_scaled)
print(df2)

for x in df2:
    print("%s has correlation %s with yield" % (df.loc[:, df_potential_columns].iloc[:, x].name, np.correlate(df2[x], potential_y)))

Index(['Yield', 'Year', 'Temperature', 'Precipitation', 'Solar Radiation',
       'Soil class', 'CEC', 'Organic matter', 'pH', 'Clay', 'Silt', 'Sand',
       'Area'],
      dtype='object')
             0    1         2         3         4         5         6   \
0      0.457460  0.0  0.361879  0.577475  0.410389  0.049550  0.811016   
1      0.441596  0.0  0.361879  0.577475  0.410389  0.049550  0.811016   
2      0.402939  0.0  0.361879  0.577475  0.410389  0.049550  0.811016   
3      0.435585  0.0  0.361879  0.577475  0.410389  0.049550  0.811016   
4      0.364699  0.0  0.361879  0.577475  0.410389  0.049550  0.811016   
5      0.388912  0.0  0.361879  0.577475  0.410389  0.049550  0.811016   
6      0.403440  0.0  0.361879  0.577475  0.410389  0.049550  0.811016   
7      0.354012  0.0  0.361879  0.577475  0.410389  0.049550  0.811016   
8      0.418803  0.0  0.361879  0.577475  0.410389  0.049550  0.811016   
9      0.421558  0.0  0.361879  0.577475  0.410389  0.049550  0.811016 

/home/kafi/.local/lib/python3.5/site-packages/ipykernel_launcher.py:24: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [ ]:
# PREVIOUSLY USED ONEHOT ENCODER

from sklearn.preprocessing import LabelEncoder
from pandas import Series
le = LabelEncoder()
integer_encoded = le.fit_transform(df.Variety)
# le.classes_


from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

print(onehot_encoded)

# df['VarietyEncode'] = onehot_encoded
# print(pd.get_dummies(df.Variety))
# print(df.Variety)
    
from numpy import argmax
inverted = le.inverse_transform([argmax(onehot_encoded[300, :])])
print([argmax(onehot_encoded[300, :])])
verted = le.transform(['V150834'])
print(inverted)
# inverted = le.inverse_transform([df.])
# print(inverted)

In [ ]:
# OPTIONAL USED FROM CONVERTING EACH VARIETY TO A ONE HOT

def convert_variety_to_one_hot(variety):
    return onehot_encoded[le.transform([variety])]

In [ ]:
# OPTIONAL APPLY ABOVE FUNCTION TO REPLACE EACH ROW IN VARIETY COLUMN WITH ONE HOT REPRESENTATION

df.Variety = df.Variety.apply(lambda val: convert_variety_to_one_hot(val))

# OPTIONAL RESHAPE
df.Variety = df.Variety.apply(lambda val: val.reshape(174))

In [ ]:
# CURRENTLY NECESSARY IF: USING 174 ADDITIONAL VARIETY COLUMNS METHOD

# THIS IS A DIFFERENT APPROACH TO THE ABOVE FOUR CELLS, WHERE WE HAVE 174 ADDITIONAL FEATURE COLUMNS
# EACH WITH A 0 (IF IT IS NOT OF THAT VARIETY) OR A 1 (IF IT IS OF THAT VARIETY)

# print(df)
dummies = pd.get_dummies(df.Variety)
# print(dummies)
df = pd.concat([df, dummies], axis=1)
print(df)

In [ ]:
# ORIGINAL COLUMNS! WE WANT TO PICK ONES THAT ARE GOOD FOR OUR LEARNING ALGORITHM AND DROP THE REST IN THE
# FOLLOWING CELL

for col in df.columns:
    print(col, type(df[col][0]))

In [2]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

df = df.drop(['Experiment', 'Location', 'Planting date', 'Check Yield', 'Yield difference', 'Latitude', 'Longitude', 'Variety', 'PI'], axis=1)
df['YieldBucket'] = pd.Series(pd.qcut(df.Yield, q=3, labels=["high", "medium", "low"]))


In [ ]:
print(df.YieldBucket)

In [3]:
# LET US ALSO MAKE SURE THERE ARE NO NAN IN THE DATA
print("We expect to be %s nan values and there actually are %s nan values\n" % (0, np.sum(df.isnull().sum())))
# AFTER COLUMNS, MAKE SURE NO SKETCHY ONES
for col in df.columns:
    print(col, type(df[col][0]))    

We expect to be 0 nan values and there actually are 0 nan values

Yield <class 'numpy.float64'>
Year <class 'numpy.int64'>
Temperature <class 'numpy.float64'>
Precipitation <class 'numpy.float64'>
Solar Radiation <class 'numpy.int64'>
Soil class <class 'numpy.int64'>
CEC <class 'numpy.float64'>
Organic matter <class 'numpy.float64'>
pH <class 'numpy.float64'>
Clay <class 'numpy.float64'>
Silt <class 'numpy.float64'>
Sand <class 'numpy.float64'>
Area <class 'numpy.float64'>
YieldBucket <class 'str'>


In [4]:
def scale(dataframe):
    potential = dataframe.values
    min_max_scaler = preprocessing.StandardScaler()
    x_scaled = min_max_scaler.fit_transform(potential)
    return pd.DataFrame(x_scaled)

In [7]:
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT

# feature_columns = ['Variety', 'Solar Radiation', 'Temperature', 'Precipitation', 'Location', 'Clay', 'Silt', 'Sand', 'pH', 'Precipitation', 'CEC', 'Soil class']

X = df.drop(['Yield', 'YieldBucket'], axis=1)

print(X.columns)
# X = df.loc[:, feature_columns]

y = df.Yield

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, train_size = 0.1, random_state = 42)

# train_visual, zz, zzz, zzzz = train_test_split(df, y, test_size=0.2, train_size=0.8, random_state = 42)

from sklearn import preprocessing
from sklearn import utils

Index(['Year', 'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class',
       'CEC', 'Organic matter', 'pH', 'Clay', 'Silt', 'Sand', 'Area'],
      dtype='object')


In [14]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
estimator = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=0)
# estimator = SVR(kernel="linear")
# estimator = 
selector = RFECV(estimator, step=1, cv=5, verbose=1)
selector = selector.fit(X_train, y_train)
selector.support_ 
selector.ranking_

Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting est

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# UNUSED SCALING

from sklearn.preprocessing import StandardScaler
standard_scalar = StandardScaler()
x_std = standard_scalar.fit_transform(X_train)
print(x_std.shape)

In [ ]:
# VISUALIZE FEATURE DISTRIBUTIONS OPTIONALLY

import matplotlib.pyplot as plt
X_train[X_train.dtypes[(X_train.dtypes=="float64")|(X_train.dtypes=="int64")]
                        .index.values].hist(figsize=[11,11])

In [8]:
# IT COULDN'T HURT TO SEE THE FINAL SHAPES

# print(X_train.describe())
# print(X_train.head())
# print(y_train.describe())
# print(y_train.head())
print(X_train.shape)
print(y_train.shape)

(8203, 12)
(8203,)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

errors = np.absolute(((preds - y_test) / y_test) * 100)
# erros = 
print(errors)
print(np.mean(errors))

In [ ]:
# GET OUTPUT OF FEATURE IMPORTANCE

feature_importances = regr.feature_importances_
feature_importances = pd.Series(feature_importances)
feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
for index, row in feature_importance_df.iterrows():
    print(row['feature'], 'has importance: ', row['feature_importance'])
# for feature_importance in regr.feature_importances_:
    
print(errors.describe())

In [ ]:
# THIS WILL ONLY WORK WITH THE BUCKET METHOD

from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor

from sklearn.feature_selection import RFECV

classifiers = [
    svm.SVR(),
    MLPRegressor(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.ARDRegression(),
#     linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]




# estimator = svm.SVR(kernel="linear")

# selector = RFECV(estimator, step=1, cv=5, verbose=1)
# selector = selector.fit(X_train, y_train)
# selector.support_ 
# # array([ True,  True,  True,  True,  True,
# #         False, False, False, False, False], dtype=bool)
# selector.ranking_
# # array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])


#     print(np.sum(preds - y_test))
#     print(clf.predict(X_test),'\n')
#     print(y_test)
#     print('accuracy score:', accuracy_score(y_test, clf.predict(X_test)), '\n')


In [ ]:
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
    print(np.mean(errors))
    print(errors.describe())

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import tree
classifier = tree.DecisionTreeClassifier()
classifier.fit(X_train, y_train)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
from sklearn.metrics import accuracy_score
for item in classifiers:
    print(item)
    clf = item
    clf.fit(scale(X_train), y_train)
    preds = clf.predict(scale(X_test))
    print(accuracy_score(y_test, preds))
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(np.mean(errors))

In [ ]:
print(scale(X_train).head)

In [ ]:
print(potential_X)

In [ ]:
potential_X = X_train.values

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(potential_X)
potential_y = y
df2 = pd.DataFrame(x_scaled)

In [ ]:
print(df2)
print(x_scaled)
print(potential_X.shape)

In [ ]:

from keras.models import Sequential
from keras.layers import Dense
from sklearn import datasets
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
# load pima price dataset
dataset = datasets.load_boston()
# split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]
# create model
model = Sequential()
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
# model.fit(X, y, epochs=1000, verbose=0)

# Compile model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(df2, y_train, epochs=150, batch_size=10)
# evaluate the model
scores = model.evaluate(X_train, y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# # convert an array of values into a dataset matrix
# def create_dataset(dataset, look_back=1):
# 	dataX, dataY = [], []
# 	for i in range(len(dataset)-look_back-1):
# 		a = dataset[i:(i+look_back), 0]
# 		dataX.append(a)
# 		dataY.append(dataset[i + look_back, 0])
# 	return numpy.array(dataX), numpy.array(dataY)
# # fix random seed for reproducibility
# numpy.random.seed(7)
# # load the dataset
# # dataframe = read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv', engine='python', skipfooter=3)
# dataset = dataframe.values
# dataset = dataset.astype('float32')
# # normalize the dataset
# scaler = MinMaxScaler(feature_range=(0, 1))
# dataset = scaler.fit_transform(dataset)
# # split into train and test sets
# train_size = int(len(dataset) * 0.67)
# test_size = len(dataset) - train_size
# train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# # reshape into X=t and Y=t+1
look_back = 12
# trainX, trainY = create_dataset(train, look_back)
# testX, testY = create_dataset(test, look_back)
# # reshape input to be [samples, time steps, features]
# trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(8203, look_back)))
model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()